In [1]:
!pip install konlpy

In [2]:
from konlpy.tag import Okt
import re
okt = Okt()

In [3]:
from tqdm import tqdm_notebook
import numpy as np
from keras.layers import LSTM, Activation, Dropout, Dense, Input, Bidirectional
from keras.layers import Embedding
from keras.models import Model
from keras import layers
from keras.callbacks import EarlyStopping
import keras
from keras.models import Sequential

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences


In [4]:
import pandas as pd

csv = pd.read_csv("CrawlingSet10000_1.csv", index_col=0, encoding='cp949')
csv

,Title,Singer,Lyrics,Date,Genre
Id,,,,,
35590600,읊어본 너의 두 눈엔,그네,읊어본 너의 두 눈엔 슬픔이 가득해서 보듬어 주고픈 이 내 맘 머뭇거리는데 읊조린 ...,2022,발라드
35585693,18 (feat. Paloalto),성수민 (SEONGSOO),Baby Would you come and just dance with me Wou...,2022,R&B/Soul
35583358,빈자리,한영주,울지마 울지마 울지마 떠나간 빈자리에 앉아 마지막 순간조차 지켜주지 못했던 떠나간 ...,2022,발라드
35590685,헤어짐,김대훈,그렇게 시간이 흘러 오늘까지 와버렸네요 언제까지 그날에 갇혀 살아가야 해야하나요 한...,2022,발라드
35590378,Blended Rap,Unsigned nino (언사인드 니노),You can't mess with me. My rap is Michael Tyso...,2022,랩/힙합
...,...,...,...,...,...
36922232,THE RHYTHM OF JUSTICE,박지훈 (D.A),"We march in lockstep, Advancing into death, ou...",2023,록/메탈
36917725,도시의 별빛,문득,이른 아침 눈을 떴을 때부터 너를 그려 하루 종일 우리는 함께해 어떡하면 언제나 서...,2023,발라드
36918436,다시 누군가를 사랑한다면,한경일,니가 내게 했던 말 넌 기억나니 더 좋은 사람 만나라면서 이제 내가 싫어졌다 말을 ...,2023,발라드


In [5]:
csv.head()

,Title,Singer,Lyrics,Date,Genre
Id,,,,,
35590600,읊어본 너의 두 눈엔,그네,읊어본 너의 두 눈엔 슬픔이 가득해서 보듬어 주고픈 이 내 맘 머뭇거리는데 읊조린 ...,2022,발라드
35585693,18 (feat. Paloalto),성수민 (SEONGSOO),Baby Would you come and just dance with me Wou...,2022,R&B/Soul
35583358,빈자리,한영주,울지마 울지마 울지마 떠나간 빈자리에 앉아 마지막 순간조차 지켜주지 못했던 떠나간 ...,2022,발라드
35590685,헤어짐,김대훈,그렇게 시간이 흘러 오늘까지 와버렸네요 언제까지 그날에 갇혀 살아가야 해야하나요 한...,2022,발라드
35590378,Blended Rap,Unsigned nino (언사인드 니노),You can't mess with me. My rap is Michael Tyso...,2022,랩/힙합


In [6]:
print(csv.isnull().sum())

Title     0
Singer    0
Lyrics    0
Date      0
Genre     0
dtype: int64


In [7]:
# cate = pd.get_dummies(data = csv['Unnamed: 5'])
# cate['일상생활'] = cate['일상']|cate['생활']
# cate = cate.drop(['일상','생활'],axis = 1)
# cate = cate.astype(int)
# cate

In [8]:
# csv.drop(columns = ["Unnamed: 5"], axis = 1, inplace = True)


In [9]:
# csv = pd.concat([csv,cate], axis = 1)

In [10]:
# csv.drop(columns=["노래제목","가수"],axis=1,inplace=True)
# csv


In [11]:
lylics = csv["Lyrics"].copy()

csv['Lyrics'] = csv['Lyrics'].str.replace('[^\w\s]', '')

def remove_english(text):
    # 영어 알파벳과 \n을 찾아서 삭제
    result = re.sub(r'[a-zA-Z,.!?]', '', text)
    result = result.replace('\n', ' ')
    return result

lylics_list=[]
for lylic in lylics:
    lylics_list.append(remove_english(lylic))

lylics_list[:5]

<ipython-input-11-8df67e821ea5>:3: FutureWarning: The default value of regex will change from True to False in a future version.
  csv['Lyrics'] = csv['Lyrics'].str.replace('[^\w\s]', '')


['읊어본 너의 두 눈엔 슬픔이 가득해서 보듬어 주고픈 이 내 맘 머뭇거리는데 읊조린 너의 입가에 두 귀를 기울이니 그리도 데인 맘 무색히 그 이름 부르는데 아아 울지 마라 그리 서럽게 울지 마라 날아가던 소쩍새 날갯짓 멈춘다 아아 어찌 그리 너를 쉼 없이 울리는가 곁이라면 내 곁이라면 웃음만 주었을 것을 훑어본 나의 마음엔 한 사람뿐이라서 외면을 위한 고갯짓은 아무 의미가 없다오 아아 울지 마라 그리 서럽게 울지 마라 날아가던 소쩍새 날갯짓 멈춘다 아아 어찌 그리 너를 쉼 없이 울리는가 곁이라면 내 곁이라면 웃음만 주었을 것을 그 모습마저 아름다운 너를 아아 스쳐가는 결코 머무르지 못하는 인연으로 남기길 바라던 맘였는데 아아 나는 울어도 좋으니 악연으로 남아도 좋으니 네 울음만 멈춰다오',
 '                                  지금 이 시간에 너넨 뭘 하고 있을련지 벌써 시간은 여기에 있는데 나는 계속 뒤쳐진 것 같지 이게 아닌 것만 같아 잘못되어버릴 것 같다는 생각이 요즘 많이 든 것 같아 그래 너네도 이럴 때 있잖아 가만히 열여덟인채로 남아있고 싶긴 하지만 그럴 수는 없잖아 바람처럼 왔다 이슬처럼 갈 순 없잖아 우리 시간이 남아있다면 무서워도 나 갈래 아무리 넘어져도 나는 갈래                                   넌 아직 어리고 할 게 많아 아직은 다 과정이고 숫자에 얽메이면 너의 감정은 그냥 소비고 이 행위를 돈벌이로 생각하기 전에 멋진 곡을 차곡차곡 쌓아가다 보면 결국 빛 볼거야 다 알 것 같다가도 모르는게 많다는 걸 깨닫게 되는것이 삶이라서 나한텐 짜릿하더라고 너랑 한 잔 하고 싶지만 안되니까 이 곡을 통해 잔 부딪혀 사람들이 내 재능을 몰라봤을 때 답답하고 화도 났었지 뭐 화내서 뭐해 그 에너지로 곡을 더 만드는 게 도움 돼 창작욕이 넘치면 언젠가 그게 돈도 돼 이 트랙도 의미있는 움직임 진심이 담겨 있으니 위기도 기회로 여기고 만들어가자 부지런히 나도 한땐 루저였지 하지만 이젠 연승해 남들이 보

In [12]:
# for i in range(len(lylics)):
#     lylics[i] = okt.nouns(lylics[i])
#     print(i,end=" ")

In [13]:
# from konlpy.tag import Twitter

# def keyword_extractor(tagger, text):
#     tokens = tagger.phrases(text)
#     tokens = [ token for token in tokens if len(token) > 1 ] # 한 글자인 단어는 제외
#     count_dict = [(token, text.count(token)) for token in tokens ]
#     ranked_words = sorted(count_dict, key=lambda x:x[1], reverse=True)[:10]
#     return [ keyword for keyword, freq in ranked_words ]

# i = 0
# lylics_list_okt = []
# for lylics in lylics_list:
#     twit = Twitter()
#     lylics_list_okt.append(keyword_extractor(twit, lylics))
#     print(i, end=" ")
#     i += 1


In [14]:
# lylics_list_okt[:5]

In [15]:
csv

,Title,Singer,Lyrics,Date,Genre
Id,,,,,
35590600,읊어본 너의 두 눈엔,그네,읊어본 너의 두 눈엔 슬픔이 가득해서 보듬어 주고픈 이 내 맘 머뭇거리는데 읊조린 ...,2022,발라드
35585693,18 (feat. Paloalto),성수민 (SEONGSOO),Baby Would you come and just dance with me Wou...,2022,R&B/Soul
35583358,빈자리,한영주,울지마 울지마 울지마 떠나간 빈자리에 앉아 마지막 순간조차 지켜주지 못했던 떠나간 ...,2022,발라드
35590685,헤어짐,김대훈,그렇게 시간이 흘러 오늘까지 와버렸네요 언제까지 그날에 갇혀 살아가야 해야하나요 한...,2022,발라드
35590378,Blended Rap,Unsigned nino (언사인드 니노),You cant mess with me My rap is Michael Tyson ...,2022,랩/힙합
...,...,...,...,...,...
36922232,THE RHYTHM OF JUSTICE,박지훈 (D.A),We march in lockstep Advancing into death our ...,2023,록/메탈
36917725,도시의 별빛,문득,이른 아침 눈을 떴을 때부터 너를 그려 하루 종일 우리는 함께해 어떡하면 언제나 서...,2023,발라드
36918436,다시 누군가를 사랑한다면,한경일,니가 내게 했던 말 넌 기억나니 더 좋은 사람 만나라면서 이제 내가 싫어졌다 말을 ...,2023,발라드


In [16]:
csv["remove_english_lylics"] = lylics_list

In [17]:

keywords=[]
# # keywords_1 = ["사랑", "그대", "안녕", "너무", "다시", "내가", "너를", "지금", "나를", "사랑해"]
# keywords_1 = ["사랑", "그대", "안녕", "너무", "다시", "지금", "사랑해"]
# # keywords_2 = ["내가", "너를", "나를", "우리", "니가", "다시", "이별", "정말", "혼자", "너무"]
# keywords_2 = ["우리", "다시", "이별", "정말", "혼자", "너무"]
# # keywords_3 = ["우리", "함께", "나의", "너와", "서로", "보고", "우린", "다시", "친구야", "내가"]
# keywords_3 = ["우리", "함께", "서로", "보고", "우린", "다시", "친구야"]
# # keywords_4 = ["엄마", "내가", "나의", "이제", "사람", "아빠", "우리", "울어", "가족", "아버지"]
# keywords_4 = ["엄마", "이제", "사람", "아빠", "우리", "울어", "가족", "아버지"]
# # keywords_5 = ["날아올라", "너를", "나를", "높이", "길을", "가게", "내가", "꿈을", "꿈", "멀리"]
# keywords_5 = ["날아올라", "높이", "길을", "가게", "꿈을", "꿈", "멀리"]
# # keywords_6 = ["내가", "거야", "나의", "너의", "함께", "나를", "다시", "이태원", "괜찮아", "성장"]
# keywords_6 = ["거야", "함께", "다시", "이태원", "괜찮아", "성장"]
# # keywords_7 = ["다시", "혼자", "네가", "나를", "챔피언", "그리워", "감정", "우울하다", "내가", "우울해"]
# keywords_7 = ["다시", "혼자", "챔피언", "그리워", "감정", "우울하다", "우울해"]
# # keywords_8 = ["좋아", "떠날래", "나의", "나도", "따라", "아침", "싶어", "환경", "커다란", "내겐"]
# keywords_8 = ["좋아", "떠날래", "따라", "아침", "싶어", "환경", "커다란", "내겐"]
# # keywords_9 = ["나는", "멋대로", "먹고", "죄송합니다", "일상", "달콤한", "행복을", "꿈을", "기분좋아지는", "싶은"]
# keywords_9 = ["멋대로", "먹고", "죄송합니다", "일상", "달콤한", "행복을", "꿈을", "기분좋아지는", "싶은"]

keywords_1 = ["사랑", "그대", "안녕", "너무", "다시", "지금"]
keywords_2 = ["우리", "다시", "이별", "정말", "혼자", "너무"]
keywords_3 = ["우리", "함께", "서로", "보고", "다시", "친구야"]
keywords_4 = ["엄마", "이제", "사람", "아빠", "우리", "울어", "가족", "아버지"]
keywords_5 = ["날아", "높이", "길을", "가게", "꿈을", "꿈", "멀리"]
keywords_6 = ["좋아", "떠날래", "따라", "아침", "싶어", "환경", "커다란", "내겐"]

keywords.append(keywords_1)
keywords.append(keywords_2)
keywords.append(keywords_3)
keywords.append(keywords_4)
keywords.append(keywords_5)
keywords.append(keywords_6)

def classify_lyrics(lyrics):


    # 가사 분류
    if any(keyword in lyrics for keyword in keywords_1):
        return 1
    elif any(keyword in lyrics for keyword in keywords_2):
        return 2
    elif any(keyword in lyrics for keyword in keywords_3):
        return 3
    elif any(keyword in lyrics for keyword in keywords_4):
        return 4
    elif any(keyword in lyrics for keyword in keywords_5):
        return 5
    elif any(keyword in lyrics for keyword in keywords_6):
        return 6
    else:
        return 0  # 해당되는 분류가 없는 경우

# '가사' 칼럼의 각 행에 대해 classify_lyrics 함수를 적용하여 '분류' 칼럼을 생성
csv['label'] = csv['remove_english_lylics'].apply(classify_lyrics)

In [18]:
csv

,Title,Singer,Lyrics,Date,Genre,remove_english_lylics,label
Id,,,,,,,
35590600,읊어본 너의 두 눈엔,그네,읊어본 너의 두 눈엔 슬픔이 가득해서 보듬어 주고픈 이 내 맘 머뭇거리는데 읊조린 ...,2022,발라드,읊어본 너의 두 눈엔 슬픔이 가득해서 보듬어 주고픈 이 내 맘 머뭇거리는데 읊조린 ...,4
35585693,18 (feat. Paloalto),성수민 (SEONGSOO),Baby Would you come and just dance with me Wou...,2022,R&B/Soul,지금 이 시간에 너넨 ...,1
35583358,빈자리,한영주,울지마 울지마 울지마 떠나간 빈자리에 앉아 마지막 순간조차 지켜주지 못했던 떠나간 ...,2022,발라드,울지마 울지마 울지마 떠나간 빈자리에 앉아 마지막 순간조차 지켜주지 못했던 떠나간 ...,0
35590685,헤어짐,김대훈,그렇게 시간이 흘러 오늘까지 와버렸네요 언제까지 그날에 갇혀 살아가야 해야하나요 한...,2022,발라드,그렇게 시간이 흘러 오늘까지 와버렸네요 언제까지 그날에 갇혀 살아가야 해야하나요 한...,2
35590378,Blended Rap,Unsigned nino (언사인드 니노),You cant mess with me My rap is Michael Tyson ...,2022,랩/힙합,' ' - ' ' ...,1
...,...,...,...,...,...,...,...
36922232,THE RHYTHM OF JUSTICE,박지훈 (D.A),We march in lockstep Advancing into death our ...,2023,록/메탈,...,0
36917725,도시의 별빛,문득,이른 아침 눈을 떴을 때부터 너를 그려 하루 종일 우리는 함께해 어떡하면 언제나 서...,2023,발라드,이른 아침 눈을 떴을 때부터 너를 그려 하루 종일 우리는 함께해 어떡하면 언제나 서...,1
36918436,다시 누군가를 사랑한다면,한경일,니가 내게 했던 말 넌 기억나니 더 좋은 사람 만나라면서 이제 내가 싫어졌다 말을 ...,2023,발라드,니가 내게 했던 말 넌 기억나니 더 좋은 사람 만나라면서 이제 내가 싫어졌다 말을 ...,1


In [19]:
cate = pd.get_dummies(data = csv['label'])
cate = cate.astype(int)
csv = pd.concat([csv,cate], axis = 1)

In [20]:
csv

,Title,Singer,Lyrics,Date,Genre,remove_english_lylics,label,0,1,2,3,4,5,6
Id,,,,,,,,,,,,,,
35590600,읊어본 너의 두 눈엔,그네,읊어본 너의 두 눈엔 슬픔이 가득해서 보듬어 주고픈 이 내 맘 머뭇거리는데 읊조린 ...,2022,발라드,읊어본 너의 두 눈엔 슬픔이 가득해서 보듬어 주고픈 이 내 맘 머뭇거리는데 읊조린 ...,4,0,0,0,0,1,0,0
35585693,18 (feat. Paloalto),성수민 (SEONGSOO),Baby Would you come and just dance with me Wou...,2022,R&B/Soul,지금 이 시간에 너넨 ...,1,0,1,0,0,0,0,0
35583358,빈자리,한영주,울지마 울지마 울지마 떠나간 빈자리에 앉아 마지막 순간조차 지켜주지 못했던 떠나간 ...,2022,발라드,울지마 울지마 울지마 떠나간 빈자리에 앉아 마지막 순간조차 지켜주지 못했던 떠나간 ...,0,1,0,0,0,0,0,0
35590685,헤어짐,김대훈,그렇게 시간이 흘러 오늘까지 와버렸네요 언제까지 그날에 갇혀 살아가야 해야하나요 한...,2022,발라드,그렇게 시간이 흘러 오늘까지 와버렸네요 언제까지 그날에 갇혀 살아가야 해야하나요 한...,2,0,0,1,0,0,0,0
35590378,Blended Rap,Unsigned nino (언사인드 니노),You cant mess with me My rap is Michael Tyson ...,2022,랩/힙합,' ' - ' ' ...,1,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36922232,THE RHYTHM OF JUSTICE,박지훈 (D.A),We march in lockstep Advancing into death our ...,2023,록/메탈,...,0,1,0,0,0,0,0,0
36917725,도시의 별빛,문득,이른 아침 눈을 떴을 때부터 너를 그려 하루 종일 우리는 함께해 어떡하면 언제나 서...,2023,발라드,이른 아침 눈을 떴을 때부터 너를 그려 하루 종일 우리는 함께해 어떡하면 언제나 서...,1,0,1,0,0,0,0,0
36918436,다시 누군가를 사랑한다면,한경일,니가 내게 했던 말 넌 기억나니 더 좋은 사람 만나라면서 이제 내가 싫어졌다 말을 ...,2023,발라드,니가 내게 했던 말 넌 기억나니 더 좋은 사람 만나라면서 이제 내가 싫어졌다 말을 ...,1,0,1,0,0,0,0,0


In [21]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report

# 특징 벡터화 (CountVectorizer 사용)
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(csv['Lyrics'])

# 레이블
y = csv[cate.columns.tolist()]

# 학습 데이터와 테스트 데이터로 나누기
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

# 랜덤 포레스트 모델 생성 및 학습
rf_model = RandomForestClassifier(n_estimators=50, max_depth=3, random_state=42)
rf_model.fit(X_train, y_train)

# 테스트 데이터에 대한 예측
y_pred = rf_model.predict(X_test)

# 정확도 출력
print("Accuracy:", accuracy_score(y_test, y_pred))

# 기타 평가 지표 출력
print(classification_report(y_test, y_pred))

(6842, 118567)
(6842, 7)
(2933, 118567)
(2933, 7)
Accuracy: 0.6992840095465394
              precision    recall  f1-score   support

           0       0.00      0.00      0.00       324
           1       0.70      1.00      0.82      2051
           2       0.00      0.00      0.00       250
           3       0.00      0.00      0.00        88
           4       0.00      0.00      0.00        79
           5       0.00      0.00      0.00        87
           6       0.00      0.00      0.00        54

   micro avg       0.70      0.70      0.70      2933
   macro avg       0.10      0.14      0.12      2933
weighted avg       0.49      0.70      0.58      2933
 samples avg       0.70      0.70      0.70      2933



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [22]:
from sklearn.multioutput import MultiOutputClassifier
from sklearn.naive_bayes import MultinomialNB

# 나이브 베이즈 모델 생성 및 학습
nb_model = MultiOutputClassifier(MultinomialNB())
nb_model.fit(X_train, y_train)

# 테스트 데이터에 대한 예측
y_pred = nb_model.predict(X_test)

# 정확도 출력
print("Accuracy:", accuracy_score(y_test, y_pred))

# 기타 평가 지표 출력
print(classification_report(y_test, y_pred))


Accuracy: 0.6505284691442209
              precision    recall  f1-score   support

           0       0.55      0.81      0.65       324
           1       0.81      0.84      0.83      2051
           2       0.19      0.14      0.16       250
           3       0.11      0.08      0.09        88
           4       0.11      0.10      0.10        79
           5       0.07      0.05      0.06        87
           6       0.07      0.06      0.06        54

   micro avg       0.68      0.70      0.69      2933
   macro avg       0.27      0.30      0.28      2933
weighted avg       0.65      0.70      0.67      2933
 samples avg       0.67      0.70      0.67      2933



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
